In [ ]:
!pip3 install openai
!pip3 install python-dotenv
!pip install transformers datasets faiss-cpu -q
!pip install opencv-python
!pip install streamlit

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

import os
print(os.system('ls'))

os.chdir(os.curdir + "/drive/MyDrive/Colab_Notebooks_LLM_2023")

Mounted at /content/drive/
0


In [ ]:
import os
import openai
from openai import OpenAI
from PIL import Image
import urllib.request
from io import BytesIO
from IPython.display import display
import time

open_ai_key_file = "openai_api_key_llm_2023.txt"
with open(open_ai_key_file, "r") as f:
  for line in f:
    OPENAI_KEY = line
    break

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = OPENAI_KEY

folder_path = "flower_images"
os.makedirs(folder_path, exist_ok=True)

client = OpenAI(api_key=OPENAI_KEY)

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )

    return response.choices[0].message.content

prompt = f"""Generate the names of 150 distinct species of flowers with no repeated names in one line.
              Seperate each name with a comma. Format like: A,B,C. And for flower names with white space,
              replace the white space with underscore _."""

response = get_completion(prompt)

In [ ]:
flower_names = response.split(',')

unique_names_set = set()
unique_flower_names = []

for name in flower_names:
  if name not in unique_names_set and len(unique_flower_names) < 100:
    unique_names_set.add(name)
    unique_flower_names.append(name)

flower_names_file = "flower_names.txt"

with open(flower_names_file, 'w') as file:
    file.write('')

unique_flower_names.sort()

with open(flower_names_file, 'w') as file:
  for name in unique_flower_names:
    file.write(name + '\n')

flower_names_file = "flower_names.txt"
# Read the flower names from the file
with open(flower_names_file, 'r') as file:
  flower_names_read = file.read().split()

In [ ]:
def generate_and_save_image(flower_name, image_number):
    response = client.images.generate(
      model="dall-e-2",
      prompt=f"Real Flower: {flower_name}.",
      size="1024x1024",
      quality="standard",
      style="natural",
      n=1,
    )

    image_url = response.data[0].url
    with urllib.request.urlopen(image_url) as url:
        img = Image.open(BytesIO(url.read()))
        img.save(os.path.join(folder_path, f"{image_number}_{flower_name}.png"))

# Generate and save images for each flower
for i, name in enumerate(flower_names_read):
    image_path = os.path.join(folder_path, f"{i}_{name}.png")

    # Check if the image already exists
    if not os.path.exists(image_path):
        try:
            generate_and_save_image(name, i)
            time.sleep(1)  # Adding delay to avoid rate limiting
        except Exception as e:
            print(f"Error generating image for {name}: {e}")

print("Image generation complete.")

Image generation complete.
